In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set paths to image and xml folders
image_base = '/content/drive/MyDrive/lesion_segmentation'
xml_base = '/content/drive/MyDrive/lesion_detection'


Mounted at /content/drive


In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Lesion class mapping
class_map = {'ex': 0, 'he': 1, 'se': 2, 'ma': 3}  # Adjust if needed

def parse_annotations(xml_folder, image_folder):
    annotations = []

    for xml_file in os.listdir(xml_folder):
        if not xml_file.endswith('.xml'):
            continue

        xml_path = os.path.join(xml_folder, xml_file)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        filename = root.find('filename').text
        image_path = os.path.join(image_folder, filename)

        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in class_map:
                continue
            class_id = class_map[class_name]

            bbox = obj.find('bndbox')
            xmin = int(float(bbox.find('xmin').text))
            ymin = int(float(bbox.find('ymin').text))
            xmax = int(float(bbox.find('xmax').text))
            ymax = int(float(bbox.find('ymax').text))

            annotations.append([image_path, xmin, ymin, xmax, ymax, class_id])

    return annotations


In [ ]:
splits = ['train', 'valid', 'test']

for split in splits:
    image_folder = os.path.join(image_base, split, 'image')
    xml_folder = os.path.join(xml_base, split)

    annots = parse_annotations(xml_folder, image_folder)
    df = pd.DataFrame(annots, columns=['image_path', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id'])
    df.to_csv(f'{split}_annotations.csv', index=False)


In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17410, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 17410 (delta 63), reused 22 (delta 22), pack-reused 17326 (from 3)
Receiving objects: 100% (17410/17410), 16.30 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (11934/11934), done.


In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 64.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import pandas as pd
import os
from PIL import Image

def convert_to_yolo_format(csv_path, output_label_dir, image_base_path):
    os.makedirs(output_label_dir, exist_ok=True)
    df = pd.read_csv(csv_path)

    for img_path in df['image_path'].unique():
        # Construct full image path
        full_img_path = img_path
        if not os.path.exists(full_img_path):
            print(f"Image not found: {full_img_path}")
            continue

        label_path = os.path.join(output_label_dir, os.path.basename(img_path).replace('.jpg', '.txt'))

        img_df = df[df['image_path'] == img_path]
        try:
            with Image.open(full_img_path) as im:
                w, h = im.size
        except Exception as e:
            print(f"Error opening image {full_img_path}: {e}")
            continue

        with open(label_path, 'w') as f:
            for _, row in img_df.iterrows():
                x_center = (row['xmin'] + row['xmax']) / 2.0 / w
                y_center = (row['ymin'] + row['ymax']) / 2.0 / h
                box_width = (row['xmax'] - row['xmin']) / w
                box_height = (row['ymax'] - row['ymin']) / h
                f.write(f"{row['class_id']} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

# Example usage
image_base_path = '/content/drive/MyDrive/lesion_segmentation/train/image'
convert_to_yolo_format('/content/train_annotations.csv', '/content/yolo_dataset/train/labels', image_base_path)
convert_to_yolo_format('/content/valid_annotations.csv', '/content/yolo_dataset/val/labels', image_base_path)
convert_to_yolo_format('/content/test_annotations.csv', '/content/yolo_dataset/test/labels', image_base_path)


In [ ]:
import shutil
from tqdm import tqdm

def copy_images_from_csv(csv_path, target_img_dir):
    os.makedirs(target_img_dir, exist_ok=True)
    df = pd.read_csv(csv_path)
    for img_path in tqdm(df['image_path'].unique()):
        try:
            shutil.copy(img_path, target_img_dir)
        except:
            print(f"Failed to copy {img_path}")

copy_images_from_csv('/content/train_annotations.csv', '/content/yolo_dataset/train/images')
copy_images_from_csv('/content/valid_annotations.csv', '/content/yolo_dataset/val/images')
copy_images_from_csv('/content/test_annotations.csv', '/content/yolo_dataset/test/images')


100%|██████████| 225/225 [01:15<00:00,  3.00it/s]


In [ ]:
yaml_content = """
path: /content/yolo_dataset
train: train/images
val: val/images
test: test/images

nc: 4
names: ['EX', 'HE', 'SE', 'MA']
"""

with open('/content/yolov5/lesion.yaml', 'w') as f:
    f.write(yaml_content)


In [ ]:
!python train.py --img 640 --batch 16 --epochs 10 --data lesion.yaml --weights yolov5s.pt --name lesion_detect


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-22 16:03:38.731916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745337819.001803    9268 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745337819.074751    9268 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [ ]:
!python val.py --weights runs/train/lesion_detect/weights/best.pt --data lesion.yaml --task test

val: data=lesion.yaml, weights=['runs/train/lesion_detect/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/yolo_dataset/test/labels.cache... 225 images, 0 backgrounds, 0 corrupt: 100% 225/225 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  75% 6/8 [00:15<00:03,  1.93s/it]Corrupt JPEG data: 40 extraneous bytes before marker 0xd9
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 8/8 [00:20<00:00,  2.53s/it]
                   all    

In [ ]:
!python val.py --weights runs/train/lesion_detect/weights/best.pt \
               --data lesion.yaml \
               --img 640 \
               --verbose


val: data=lesion.yaml, weights=['runs/train/lesion_detect/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/yolo_dataset/val/labels.cache... 149 images, 0 backgrounds, 0 corrupt: 100% 149/149 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  80% 4/5 [00:12<00:02,  2.63s/it]Corrupt JPEG data: 36 extraneous bytes before marker 0xd9
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 5/5 [00:13<00:00,  2.77s/it]
                   all        